# RCOE GenAI Apps: Colab Deployment with Ngrok

This notebook sets up and deploys two Flask applications in Google Colab with ngrok public URLs:
- AskMeChatBot.py (port 5000)
- RCOEGenAIAgents.py (port 5001)

It handles configuration and secure credentials using `.env`, `config.properties`, and an OCI `oci_api_key.pem` file placed in the working directory.

Security notes:
- Do not print secrets in output cells.
- Upload `.env` and `oci_api_key.pem` at runtime; do not embed secrets in the notebook.
- ngrok tunnels are public; rotate tokens and shut down tunnels when finished.

In [ ]:
# 1) Install Required Dependencies
import sys
import subprocess

packages = [
    'flask', 'python-dotenv', 'requests', 'oci', 'pyngrok==7.1.2'
]

for pkg in packages:
    print(f"Installing {pkg}...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print("All packages installed.")

In [ ]:
# 2) Import Libraries and Configure Environment
import os
import time
import json
import threading
import textwrap
import shutil
import signal
from pathlib import Path
from dotenv import load_dotenv
from pyngrok import ngrok, conf

WORKDIR = Path('/content/OCIGenAIBot')
WORKDIR.mkdir(parents=True, exist_ok=True)
print(f"Working directory: {WORKDIR}")

# Ensure ngrok default config doesn't reuse old tunnels in Colab sessions
conf.get_default().monitor_thread = False

In [ ]:
# 3) Load Environment Variables and Configuration Files (Upload)
from google.colab import files

print("Upload required files (.env, config.properties, oci_api_key.pem, AskMeChatBot.py, RCOEGenAIAgents.py)")
uploaded = files.upload()

# Save uploaded files into WORKDIR
for name, data in uploaded.items():
    with open(WORKDIR / name, 'wb') as f:
        f.write(data)
        print(f"Saved: {name} -> {WORKDIR/name}")

# Verify presence and set secure permissions on PEM
required_files = ['.env', 'config.properties', 'oci_api_key.pem', 'AskMeChatBot.py', 'RCOEGenAIAgents.py']
missing = [f for f in required_files if not (WORKDIR / f).exists()]
if missing:
    raise FileNotFoundError(f"Missing required files: {missing}")

# Restrict PEM permissions
pem_path = WORKDIR / 'oci_api_key.pem'
os.chmod(pem_path, 0o600)
print("PEM permissions set to 600")

# Load .env now that it's in place
load_dotenv(dotenv_path=WORKDIR / '.env')
print(".env loaded")

# Preview minimal config keys from config.properties (no secrets)
config = {}
with open(WORKDIR / 'config.properties', 'r') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            k, v = line.split('=', 1)
            config[k.strip()] = v.strip()

region = config.get('genai_region', 'us-chicago-1')
print(f"genai_region = {region}")

## Notes on Configuration and Security

- The applications read `genai_region` from `config.properties` and use the local `oci_api_key.pem` from the working folder.
- `.env` must contain: `OCI_USER`, `OCI_FINGERPRINT`, `OCI_TENANCY`, `OCI_KEY_FILE=oci_api_key.pem`.
- Do not display or print secret values in the notebook output.

In [ ]:
# 4) Set Up Ngrok (Auth Token) and Create Tunnels
from getpass import getpass

NGROK_AUTH_TOKEN = os.getenv('NGROK_AUTH_TOKEN') or getpass('Enter your ngrok auth token (input hidden): ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Ensure any previous tunnels are closed
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
    except Exception:
        pass

print("Starting tunnels for ports 5000 and 5001...")
http_tunnel_5000 = ngrok.connect(5000, bind_tls=True)
http_tunnel_5001 = ngrok.connect(5001, bind_tls=True)

print("Public URLs:")
print("AskMeChatBot:", http_tunnel_5000.public_url)
print("RCOEGenAIAgents:", http_tunnel_5001.public_url)

In [ ]:
# 5) Launch Both Flask Applications (Background Threads)
import subprocess
import sys

processes = {}

os.chdir(WORKDIR)

# Helper to start a process

def start_app(label, script, port):
    if not Path(script).exists():
        raise FileNotFoundError(f"{script} not found in {WORKDIR}")
    env = os.environ.copy()
    env['PYTHONUNBUFFERED'] = '1'
    p = subprocess.Popen([sys.executable, script], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    processes[label] = p
    print(f"Started {label} on port {port} (PID={p.pid})")

start_app('AskMeChatBot', 'AskMeChatBot.py', 5000)
start_app('RCOEGenAIAgents', 'RCOEGenAIAgents.py', 5001)

print("Both applications launched.")

In [ ]:
# 6) Stream Logs (Optional): View latest logs from both apps
import itertools

print("Streaming combined logs for ~60 seconds. Stop the cell to end early.")
start_time = time.time()

while time.time() - start_time < 60:
    for label, p in list(processes.items()):
        if p.poll() is not None:
            print(f"\n[{label}] Process exited with code {p.returncode}")
            processes.pop(label, None)
            continue
        # Non-blocking read (may not always emit)
        line = p.stdout.readline()
        if line:
            print(f"[{label}] {line}", end='')
    time.sleep(0.2)

print("Log tailing complete. Use the public URLs above to access the apps.")

In [ ]:
# 7) Utility: Show Public URLs Again and Health Check
try:
    print("AskMeChatBot URL:", http_tunnel_5000.public_url)
    print("RCOEGenAIAgents URL:", http_tunnel_5001.public_url)
    
    # Optionally ping MCP servers list
    import requests
    url_servers = http_tunnel_5001.public_url + '/mcp/servers'
    print("Fetching:", url_servers)
    r = requests.get(url_servers, timeout=10)
    print("/mcp/servers ->", r.status_code)
    if r.ok:
        print(r.json())
except Exception as e:
    print("Health check error:", e)

In [ ]:
# 8) Graceful Shutdown Helpers

def stop_all():
    print("Stopping tunnels and processes...")
    try:
        ngrok.disconnect(http_tunnel_5000.public_url)
        ngrok.disconnect(http_tunnel_5001.public_url)
        ngrok.kill()
        print("Ngrok tunnels closed.")
    except Exception as e:
        print("Ngrok shutdown error:", e)
    
    for label, p in processes.items():
        try:
            p.terminate()
            try:
                p.wait(timeout=5)
            except subprocess.TimeoutExpired:
                p.kill()
            print(f"Terminated {label} (PID={p.pid})")
        except Exception as e:
            print(f"Error stopping {label}: {e}")
    processes.clear()

print("Run stop_all() in a new cell to shut everything down when finished.")